# Exercise 3: Programming the "8-Puzzle Problem"

## A/ Problem Formulation

In [1]:
import numpy as np
import heapq #heapq will help us implement a priority queue

In [2]:
class EightPuzzle:
    """
    This class will help us instantiate our Eight Puzzle problem. 
    We have an initial state represented by a 3x3 grid which we will alter with a series of actions to reach the goal state.
    """

    def __init__(self, initialState=None, goalState=None):
        """
        This method initializes the EightPuzzle class with an initial state and a goal state. If these states are not specified 
        (no values are given when creating the problem), we assign them a predefined value.

        Args:
            initialState (numpy array): the initial configuration of the puzzle
            goalState (numpy array): the goal state we want to reach
        """
        if initialState is None:
            self.initialState=np.array([[7,2,4],[5,0,6],[8,3,1]]) #we give a default initial state unless a non-empty argument is passed
            #the empty tile is represented by 0 instead of None to avoid any complication
        else:
            self.initialState=initialState
        if goalState is None:
            self.goalState=np.array([[0,1,2],[3,4,5],[6,7,8]]) #we give a default initial state unless a non-empty argument is passed
        else:
            self.goalState=goalState

    def actions(self,state):
        """
        This method returns a list of the actions (up, down, left, right) that can be executed in a given configuration of the puzzle.

        Args:
            state (numpy array): a state from the puzzle

        Returns:
            list: a list of the possible actions the empty tile (0) can make in the given state
        """
        
        m,n=state.shape 
        actions=[]
        pos_x,pos_y=np.where(state==0) #we get the position of the empty tile (0)
        pos_x, pos_y = pos_x[0], pos_y[0]

        #we add the action to the list if it is possible based on the position of the empty tile
        if pos_x>0:
            actions.append("up")
        if pos_x<m-1:
            actions.append("down")
        if pos_y<n-1:
            actions.append("right")
        if pos_y>0:
            actions.append("left")

        return actions
    
    def move(self, state, action):
        """
        This method applies an action (up, down, left or right) to the empty tile in the given state.

        Args:
            state (numpy array): a state from the puzzle
            action (string): the action to be applied to the empty tile

        Returns:
            numpy array: the new state obtained after the action was applied
        """

        pos_x,pos_y=np.where(state==0) #we get the position of the empty tile (0)
        pos_x, pos_y = pos_x[0], pos_y[0]
        newState=np.copy(state) #the np.copy is necessary if we don't want to change the state as well

        #we apply the action to the copy of our state depending of the action given
        if action=="down":
            newState[pos_x,pos_y]=state[pos_x+1,pos_y]
            newState[pos_x+1,pos_y]=state[pos_x,pos_y]

        elif action=="up":
            newState[pos_x,pos_y]=state[pos_x-1,pos_y]
            newState[pos_x-1,pos_y]=state[pos_x,pos_y]

        elif action=="left":
            newState[pos_x,pos_y]=state[pos_x,pos_y-1]
            newState[pos_x,pos_y-1]=state[pos_x,pos_y]

        elif action=="right":
            newState[pos_x,pos_y]=state[pos_x,pos_y+1]
            newState[pos_x,pos_y+1]=state[pos_x,pos_y]
        
        else:
            print("Impossible action")
            return state
        return newState


    def goalTest(self,state):
        """
        This method tests if the given state is equal to the goal state.

        Args:
            state (numpy array): a state from the puzzle

        Returns:
            bool: is true if the state is equal to the goal state and false if not
        """
        return np.array_equal(state,self.goalState)
    
    def heuristics(self,state, heuristic=None):
        """
        This method calculates different heuristic values for a given state.

        Args:
            state (numpy array): a state from the puzzle
            heuristic (string): the heuristic method to use

        Returns:
            int: the heuristic value resulting from a certain method
        """
        if heuristic is None:
            return 0
        else:
            if heuristic=="misplaced":
                h=self.misplacedTiles(state)
                return h
            elif heuristic=="manhattan":
                h=self.manhattanDistance(state)
                return h
            elif heuristic=="h3":
                h=self.h3(state)
                return h
            else:
                print("Error, the heuristic is not valid.") #error if the string given is not any of the three heuristics defined in this class
    
    def misplacedTiles(self, state):
        """
        This method calculates the number of misplaced tiles in a given state compared to the goal state.

        Args:
            state (numpy array): a state from the puzzle
        
        Returns:
            int: the number of misplaced tiles in the given state compared to the goal state
        """
        count=0
        for i in range(state.shape[0]):
            for j in range(state.shape[1]):
                if state[i,j]!=self.goalState[i,j] and state[i,j]!=0: #we make sure we don't take into account the empty tile (0)
                    count+=1
        return count
    
    def manhattanDistance(self, state):
        """
        This method calculates the sum of the distances of the tiles from their goal position.

        Args:
            state (numpy array): a state from the puzzle
        
        Returns:
            int: the sum of the distances of the tiles from their goal position
        """
        totalDistance=0
        for i in range(state.shape[0]):
            for j in range(state.shape[1]):
                if(state[i,j]!=0): #we make sure we don't take into account the empty tile (0)
                    goal_x,goal_y=np.where(self.goalState==state[i,j]) #we get the position of the corresponding tile in the goal state
                    totalDistance+=abs(i-goal_x[0])+abs(j-goal_y[0])
        return totalDistance
    
    def h3(self, state):
        """
        This method gives the number of incorrect rows and incorrect columns in a given state compared to the goal state.

        Args:
            state (numpy array): a state from the puzzle
        
        Returns:
            int: the number of incorrect rows and incorrect columns in a given state compared to the goal state
        """
        countIncorrectRow=0
        countIncorrectColumn=0
        for i in range(state.shape[0]):
            for j in range(state.shape[1]):
                if(state[i,j]!=0): #we make sure we don't take into account the empty tile (0)
                    goal_x,goal_y=np.where(self.goalState==state[i,j]) #we get the position of the corresponding tile in the goal state
                    if i!=goal_x[0]:
                        countIncorrectRow+=1
                    if j!=goal_y[0]:
                        countIncorrectColumn+=1
        return countIncorrectRow+countIncorrectColumn

    
    def displayPuzzle(self,state):
        """
        This method displays a given state.

        Args:
            state (numpy array): a state from the puzzle
        """
        print("Current state:\n")
        for i in range(state.shape[0]):
            for j in range(state.shape[1]):
                if state[i,j]!=0:
                    print(f"{state[i,j]} ", end='')
                else:
                    print("- ", end='')
            print("\n")
        




In [3]:
class Node:
    """
    This class is used to create a node in a search tree.
    A node is composed of a state, a parent, the action taken to reach this node, the depth it is situated at and its total path cost.
    """
    def __init__(self, state,parent=None, action=None, depth=0,total_path_cost=0):
        """
        This method initializes the node.

        Args:
            state (numpy array): the state the node represents
            parent (Node): the parent node from which this node was created
            action (string): the action taken to reach this node from the parent node
            depth (int): the depth the node is situated at (also represents the backward cost g(n))
            total_path_cost (int): the total cost of this node to reach the goal state
        """
        self.state=state
        self.parent=parent
        self.action=action
        self.depth=depth
        self.total_path_cost=total_path_cost

    def __lt__(self,node):
        """
        This method compares the node to another based on their total path cost.

        Args:
            node (Node): the other we want to to compare the actual node to

        Returns:
            bool: returns true if the actual node's total path cost is lesser than the other node's total path cost, return false otherwise
        """
        return self.total_path_cost<node.total_path_cost #this method is useful for the priority queue as it knows how to compare its different elements
    
    def solutionPath(self):
        """
        This method returns an ordered list of the all actions taken to get to the actual node from the initial node.

        Returns:
            list: the list of actions taken to get from the initial node to the actual node
        """
        solution_path=[]
        node=self
        while node.parent is not None: #we retrace all the actions taken by going up the path, from parent to parent
            solution_path.append(node.action)
            node=node.parent
        solution_path.reverse() #we reverse the list so we start from the initial node and end with the actual node
        return solution_path
        

## B/Uniform Cost Search

In [4]:
def UniformCostSearch(problem):
    """
    This method performs the Uniform Cost Search algorithm to find the best path from the initial state to the goal state of the problem.
    This algorithm explores nodes in order of their path cost thanks to a priority queue, so we can get the most optimal solution.

    Args:
        problem (EightPuzzle): represents the search problem, in our context: the Eight Puzzle

    Returns:
        tuple: returns both the solution path (list of actions performed to reach the goal state from the initial state) if it exists and
               the number of visited nodes. If there is no solution, it returns None. 
    """
    node = Node(problem.initialState) #we create the root of our graph with the problem's initial state
    frontier = []
    heapq.heappush(frontier, node) #our frontier is a priority queue where the order is defined by the value of the total path cost. The first nodes are the ones with the smallest total path cost.
    visited = set() #we use a set to keep track of the visited states so we don't visit the same nodes multiple times

    while frontier: #while there are still nodes to explore, we continue
        node = heapq.heappop(frontier) #we get the first node (with the lowest cost) from the queue
        if problem.goalTest(node.state):
            return node.solutionPath(),len(visited) #if we have reached the goal state, we found the solution and we return the solution path and the number of nodes visited
        node_state=tuple(node.state.flatten())
        if node_state not in visited: #if we haven't visited the node yet, we can expand to its children
            visited.add(node_state) #the node is now visited

            for action in problem.actions(node.state): #to expand the node to all of its children, we get all the actions possible and create the new states
                newState = problem.move(node.state,action)
                if newState is not None: 
                    depth=node.depth+1
                    child = Node(newState, node, action, depth,depth) #we store the new state into a new node to add it to the graph
                    child_state = tuple(child.state.flatten())
                    if child_state not in visited: 
                        heapq.heappush(frontier, child) #if the child hasn't been visited yet, we put it in the frontier
                        """we shouldn't worry about having the same node multiple times in the frontier: it will order them depending on their total path cost
                        if they are different and if the one with the lesser cost has already been visited in the mean time, we don't expand the node with the 
                        greater cost a second time"""

    return None

Let's test the UCS algorithm with our puzzle

In [5]:
puzzle=EightPuzzle()

print("Uniform Cost Search:\n")
solutionPath, totalNodesVisited=UniformCostSearch(puzzle)

print(f"Total nodes visited: {totalNodesVisited}")
totalMoves=len(solutionPath)
print(f"Total moves: {totalMoves}")

if solutionPath is not None:
    currentState=puzzle.initialState
    puzzle.displayPuzzle(currentState)
    for action in solutionPath:
        print(f"action: {action}")
        currentState=puzzle.move(currentState,action)
        puzzle.displayPuzzle(currentState)
else:
    print("No solution found")

Uniform Cost Search:

Total nodes visited: 168672
Total moves: 26
Current state:

7 2 4 

5 - 6 

8 3 1 

action: left
Current state:

7 2 4 

- 5 6 

8 3 1 

action: up
Current state:

- 2 4 

7 5 6 

8 3 1 

action: right
Current state:

2 - 4 

7 5 6 

8 3 1 

action: down
Current state:

2 5 4 

7 - 6 

8 3 1 

action: down
Current state:

2 5 4 

7 3 6 

8 - 1 

action: left
Current state:

2 5 4 

7 3 6 

- 8 1 

action: up
Current state:

2 5 4 

- 3 6 

7 8 1 

action: right
Current state:

2 5 4 

3 - 6 

7 8 1 

action: right
Current state:

2 5 4 

3 6 - 

7 8 1 

action: up
Current state:

2 5 - 

3 6 4 

7 8 1 

action: left
Current state:

2 - 5 

3 6 4 

7 8 1 

action: left
Current state:

- 2 5 

3 6 4 

7 8 1 

action: down
Current state:

3 2 5 

- 6 4 

7 8 1 

action: right
Current state:

3 2 5 

6 - 4 

7 8 1 

action: right
Current state:

3 2 5 

6 4 - 

7 8 1 

action: down
Current state:

3 2 5 

6 4 1 

7 8 - 

action: left
Current state:

3 2 5 

6 4 1 

7 

We output the solution path found with the UCS algorithm. The optimal solution can be found in 26 moves and in total 168 672 nodes were visited to find that result. We have also reconstructed the path to reach the solution from our initial state with all the actions needed to do so.

## C/Heuristics for A* Search

### Misplaced Tiles
See this part in the EightPuzzle class.

### Manhattan Distance
See this part in the EightPuzzle class.

## D/Best-First Search

In [6]:
def BestFirstSearch(problem, heuristic):
    """
    This method performs the Best-First Search algorithm using a specified heuristic to find the optimal path 
    from the initial state to the goal state of the problem.
    This algorithm explores nodes in order of their heuristic cost (lower first) thanks to a priority queue, so we can get the most optimal solution.

    Args:
        problem (EightPuzzle): represents the search problem, in our context: the Eight Puzzle
        heuristic (string): the heuristic function to use to calculate the path cost and guide the search

    Returns:
        tuple: returns both the solution path (list of actions performed to reach the goal state from the initial state) if it exists and
               the number of visited nodes. If there is no solution, it returns None. 
    """

    #This is almost the same algorithm as UCS except for the way we calculate the total path cost
    node = Node(problem.initialState,parent=None,action=None,depth=0,total_path_cost=problem.heuristics(problem.initialState,heuristic))
    frontier = []
    heapq.heappush(frontier, node)
    visited = set()

    while frontier:
        node = heapq.heappop(frontier)
        if problem.goalTest(node.state):
            return node.solutionPath(),len(visited)
        node_state=tuple(node.state.flatten())
        if node_state not in visited:
        
            visited.add(node_state)

            for action in problem.actions(node.state):
                newState = problem.move(node.state,action)
                if newState is not None: 
                    depth=node.depth+1
                    #in Best-First Search, the total path cost is equal to the distance of the node from the goal node calculated with a heuristic function
                    path_cost=problem.heuristics(newState, heuristic)  
                    child = Node(newState, node, action, depth,path_cost)
                    child_state = tuple(child.state.flatten())
                    if child_state not in visited:
                        heapq.heappush(frontier, child)

    return None

Let's test Best-First Search with our puzzle, first with the Misplaced Tiles heuristic and then with the Manhattan Distance heuristic.

In [7]:
print("Best-First Search, Misplaced Tiles:\n")
solutionPath, totalNodesVisited=BestFirstSearch(puzzle,"misplaced")

print(f"Total nodes visited: {totalNodesVisited}")
totalMoves=len(solutionPath)
print(f"Total moves: {totalMoves}")

if solutionPath is not None:
    currentState=puzzle.initialState
    puzzle.displayPuzzle(currentState)
    for action in solutionPath:
        print(f"action: {action}")
        currentState=puzzle.move(currentState,action)
        puzzle.displayPuzzle(currentState)
else:
    print("No solution found")

Best-First Search, Misplaced Tiles:

Total nodes visited: 896
Total moves: 106
Current state:

7 2 4 

5 - 6 

8 3 1 

action: left
Current state:

7 2 4 

- 5 6 

8 3 1 

action: up
Current state:

- 2 4 

7 5 6 

8 3 1 

action: right
Current state:

2 - 4 

7 5 6 

8 3 1 

action: right
Current state:

2 4 - 

7 5 6 

8 3 1 

action: down
Current state:

2 4 6 

7 5 - 

8 3 1 

action: left
Current state:

2 4 6 

7 - 5 

8 3 1 

action: up
Current state:

2 - 6 

7 4 5 

8 3 1 

action: left
Current state:

- 2 6 

7 4 5 

8 3 1 

action: down
Current state:

7 2 6 

- 4 5 

8 3 1 

action: down
Current state:

7 2 6 

8 4 5 

- 3 1 

action: right
Current state:

7 2 6 

8 4 5 

3 - 1 

action: right
Current state:

7 2 6 

8 4 5 

3 1 - 

action: up
Current state:

7 2 6 

8 4 - 

3 1 5 

action: up
Current state:

7 2 - 

8 4 6 

3 1 5 

action: left
Current state:

7 - 2 

8 4 6 

3 1 5 

action: left
Current state:

- 7 2 

8 4 6 

3 1 5 

action: down
Current state:

8 7 2 



In [8]:
print("Best-First Search, Manhattan Distance:\n")
solutionPath, totalNodesVisited=BestFirstSearch(puzzle,"manhattan")

print(f"Total nodes visited: {totalNodesVisited}")
totalMoves=len(solutionPath)
print(f"Total moves: {totalMoves}")

if solutionPath is not None:
    currentState=puzzle.initialState
    puzzle.displayPuzzle(currentState)
    for action in solutionPath:
        print(f"action: {action}")
        currentState=puzzle.move(currentState,action)
        puzzle.displayPuzzle(currentState)
else:
    print("No solution found")

Best-First Search, Manhattan Distance:

Total nodes visited: 280
Total moves: 66
Current state:

7 2 4 

5 - 6 

8 3 1 

action: down
Current state:

7 2 4 

5 3 6 

8 - 1 

action: right
Current state:

7 2 4 

5 3 6 

8 1 - 

action: up
Current state:

7 2 4 

5 3 - 

8 1 6 

action: up
Current state:

7 2 - 

5 3 4 

8 1 6 

action: left
Current state:

7 - 2 

5 3 4 

8 1 6 

action: left
Current state:

- 7 2 

5 3 4 

8 1 6 

action: down
Current state:

5 7 2 

- 3 4 

8 1 6 

action: right
Current state:

5 7 2 

3 - 4 

8 1 6 

action: down
Current state:

5 7 2 

3 1 4 

8 - 6 

action: left
Current state:

5 7 2 

3 1 4 

- 8 6 

action: up
Current state:

5 7 2 

- 1 4 

3 8 6 

action: up
Current state:

- 7 2 

5 1 4 

3 8 6 

action: right
Current state:

7 - 2 

5 1 4 

3 8 6 

action: down
Current state:

7 1 2 

5 - 4 

3 8 6 

action: left
Current state:

7 1 2 

- 5 4 

3 8 6 

action: down
Current state:

7 1 2 

3 5 4 

- 8 6 

action: right
Current state:

7 1 2 

### Comparison between both heuristics
As we can see in the outputs, Best-First Search with the Misplaced Tiles heuristic finds a solution path in 106 moves and visited 896 nodes during its search. Best-First Search with the Manhattan Distance heuristic found a solution path in 66 moves and explored 280 nodes to find it.
These results show that the Manhattan distance heuristic is much better at guiding the search towards the solution than the Misplaced Tiles heuristic. Indeed, Misplaced Tiles needs to visit 3 times more nodes than Manhattan Distance to find a solution. it also finds a less optimal solution because the number of moves required to reach it is much higher than Manhattan Distance's solution. 
Misplaced Tiles lacks precision because it doesn't consider how far the tiles are from their goal position, only if they are in the correct position or not. It is why it expands much more nodes and explores parts of the search space that are not really helpful in the search. On the contrary, Manhattan Distance calculates the number of moves needed for each tile to reach its correct position, making this heuristic more precise and closer at estimating the cost of the move. Therefore, Manhattan Distance guides the search more effectively as we can see in the number of nodes explored and the path length. We can conclude that Manhattan Distance finds the solution faster and that this solution is the most efficient.

## E/Admissible heuristic
See this part in the EightPuzzle class.

Let's test Best-First search with heuristic 3.

In [9]:
print("Best-First Search, h3:\n")
solutionPath, totalNodesVisited=BestFirstSearch(puzzle,"h3")

print(f"Total nodes visited: {totalNodesVisited}")
totalMoves=len(solutionPath)
print(f"Total moves: {totalMoves}")

if solutionPath is not None:
    currentState=puzzle.initialState
    puzzle.displayPuzzle(currentState)
    for action in solutionPath:
        print(f"action: {action}")
        currentState=puzzle.move(currentState,action)
        puzzle.displayPuzzle(currentState)
else:
    print("No solution found")

Best-First Search, h3:

Total nodes visited: 345
Total moves: 84
Current state:

7 2 4 

5 - 6 

8 3 1 

action: down
Current state:

7 2 4 

5 3 6 

8 - 1 

action: right
Current state:

7 2 4 

5 3 6 

8 1 - 

action: up
Current state:

7 2 4 

5 3 - 

8 1 6 

action: up
Current state:

7 2 - 

5 3 4 

8 1 6 

action: left
Current state:

7 - 2 

5 3 4 

8 1 6 

action: left
Current state:

- 7 2 

5 3 4 

8 1 6 

action: down
Current state:

5 7 2 

- 3 4 

8 1 6 

action: right
Current state:

5 7 2 

3 - 4 

8 1 6 

action: down
Current state:

5 7 2 

3 1 4 

8 - 6 

action: left
Current state:

5 7 2 

3 1 4 

- 8 6 

action: up
Current state:

5 7 2 

- 1 4 

3 8 6 

action: up
Current state:

- 7 2 

5 1 4 

3 8 6 

action: right
Current state:

7 - 2 

5 1 4 

3 8 6 

action: down
Current state:

7 1 2 

5 - 4 

3 8 6 

action: left
Current state:

7 1 2 

- 5 4 

3 8 6 

action: down
Current state:

7 1 2 

3 5 4 

- 8 6 

action: right
Current state:

7 1 2 

3 5 4 

8 - 6 

This heuristic is defined as the sum of the number of tiles not in the correct row and the number of tiles not in the correct column. It is admissible, because it only counts how many tiles are in the wrong row or column. As it doesn't measure any kind of distance it can never overestimate the cost to reach the goal because it doesn't take into account the number of moves needed to get the correct position. Each tile not in the correct position needs at least one move to get to the right column or row so it is a safe and minimal estimate of the actual cost.

Concerning the results of this heuristic compared to Misplaced Tiles and Manhattan Distance, Best-First Search with this heuristic finds a solution path composed of 84 moves and has visited 345 nodes to reach that solution. We have 345<896, meaning that h3 is more precise than Misplaced Tiles because we have less nodes expanded and that it also finds a solution path with less moves : 84<106. H3 is more precise than Misplaced Tiles because it gives a bit more information on how far the tile is from its correct position. In the end, it does give a more informed estimate than Misplaced Tiles because it at least takes into account the position of the tile in two dimensions. When we compare h3 with Manhattan distance, we can see that Manhattan distance is more precise: 280<345. It expands less nodes than h3 and also finds a solution path with fewest moves: 66<84. As Manhattan Distance sums the exact number of moves needed, horizontally and vertically, to reach the goal position, it gives a much more precise estimate of the cost. Thus, the search is much better guided than with h3. To conclude, Manhattan Distance is a better heuristic than h3 but h3 is a better heuristic than Misplaced Tiles.

## F/A* search

In [10]:
def AStar(problem, heuristic):
    """
    This method performs the A* Search algorithm to find the best path from the initial state to the goal state of the problem.
    This algorithm explores nodes in order of their path cost, which is the sum of the backward cost represented by depth in the context 
    and the forward cost represented by the heuristic value, thanks to a priority queue, so we can get the most optimal solution.

    Args:
        problem (EightPuzzle): represents the search problem, in our context: the Eight Puzzle
        heuristic (string): the heuristic function to use to calculate the path cost and guide the search

    Returns:
        tuple: returns both the solution path (list of actions performed to reach the goal state from the initial state) if it exists and
               the number of visited nodes. If there is no solution, it returns None. 
    """

    #This is almost the same algorithm as UCS except for the way we calculate the total path cost
    node = Node(problem.initialState,parent=None,action=None,depth=0,total_path_cost=problem.heuristics(problem.initialState,heuristic))
    frontier = []
    heapq.heappush(frontier, node)
    visited = set()

    while frontier:
        node = heapq.heappop(frontier)
        if problem.goalTest(node.state):
            return node.solutionPath(),len(visited)
        node_state=tuple(node.state.flatten())
        if node_state not in visited:
            visited.add(node_state)

            for action in problem.actions(node.state):
                newState = problem.move(node.state,action)
                if newState is not None: 
                    #in A*, the total path cost is equal to the sum of the forward distance (calculated with a heuristic function) and the backward cost 
                    #which is the depth in our case
                    depth=node.depth+1
                    h=problem.heuristics(newState, heuristic)
                    child = Node(newState, node, action, depth,depth+h)#we calculate both and add them together for the total path cost
                    child_state = tuple(child.state.flatten())
                    if child_state not in visited:
                        heapq.heappush(frontier, child)

    return None

Let's test the A* algorithm on our puzzle, first with the Misplaced Tiles heuristic and then with the Manhattan Distance heuristic.

In [11]:
puzzle=EightPuzzle()
print("A*, Misplaced Tiles:\n")
solutionPath, totalNodesVisited=AStar(puzzle,"misplaced")

print(f"Total nodes visited: {totalNodesVisited}")
totalMoves=len(solutionPath)
print(f"Total moves: {totalMoves}")

if solutionPath is not None:
    currentState=puzzle.initialState
    puzzle.displayPuzzle(currentState)
    for action in solutionPath:
        print(f"action: {action}")
        currentState=puzzle.move(currentState,action)
        puzzle.displayPuzzle(currentState)
else:
    print("No solution found")

A*, Misplaced Tiles:

Total nodes visited: 31605
Total moves: 26
Current state:

7 2 4 

5 - 6 

8 3 1 

action: left
Current state:

7 2 4 

- 5 6 

8 3 1 

action: up
Current state:

- 2 4 

7 5 6 

8 3 1 

action: right
Current state:

2 - 4 

7 5 6 

8 3 1 

action: down
Current state:

2 5 4 

7 - 6 

8 3 1 

action: down
Current state:

2 5 4 

7 3 6 

8 - 1 

action: left
Current state:

2 5 4 

7 3 6 

- 8 1 

action: up
Current state:

2 5 4 

- 3 6 

7 8 1 

action: right
Current state:

2 5 4 

3 - 6 

7 8 1 

action: right
Current state:

2 5 4 

3 6 - 

7 8 1 

action: up
Current state:

2 5 - 

3 6 4 

7 8 1 

action: left
Current state:

2 - 5 

3 6 4 

7 8 1 

action: left
Current state:

- 2 5 

3 6 4 

7 8 1 

action: down
Current state:

3 2 5 

- 6 4 

7 8 1 

action: right
Current state:

3 2 5 

6 - 4 

7 8 1 

action: right
Current state:

3 2 5 

6 4 - 

7 8 1 

action: down
Current state:

3 2 5 

6 4 1 

7 8 - 

action: left
Current state:

3 2 5 

6 4 1 

7 -

In [12]:
print("A*, Manhattan Distance:\n")
solutionPath, totalNodesVisited=AStar(puzzle,"manhattan")

print(f"Total nodes visited: {totalNodesVisited}")
totalMoves=len(solutionPath)
print(f"Total moves: {totalMoves}")

if solutionPath is not None:
    currentState=puzzle.initialState
    puzzle.displayPuzzle(currentState)
    for action in solutionPath:
        print(f"action: {action}")
        currentState=puzzle.move(currentState,action)
        puzzle.displayPuzzle(currentState)
else:
    print("No solution found")

A*, Manhattan Distance:

Total nodes visited: 2957
Total moves: 26
Current state:

7 2 4 

5 - 6 

8 3 1 

action: left
Current state:

7 2 4 

- 5 6 

8 3 1 

action: up
Current state:

- 2 4 

7 5 6 

8 3 1 

action: right
Current state:

2 - 4 

7 5 6 

8 3 1 

action: down
Current state:

2 5 4 

7 - 6 

8 3 1 

action: down
Current state:

2 5 4 

7 3 6 

8 - 1 

action: left
Current state:

2 5 4 

7 3 6 

- 8 1 

action: up
Current state:

2 5 4 

- 3 6 

7 8 1 

action: right
Current state:

2 5 4 

3 - 6 

7 8 1 

action: right
Current state:

2 5 4 

3 6 - 

7 8 1 

action: up
Current state:

2 5 - 

3 6 4 

7 8 1 

action: left
Current state:

2 - 5 

3 6 4 

7 8 1 

action: left
Current state:

- 2 5 

3 6 4 

7 8 1 

action: down
Current state:

3 2 5 

- 6 4 

7 8 1 

action: right
Current state:

3 2 5 

6 - 4 

7 8 1 

action: right
Current state:

3 2 5 

6 4 - 

7 8 1 

action: down
Current state:

3 2 5 

6 4 1 

7 8 - 

action: left
Current state:

3 2 5 

6 4 1 

7

### Comparison between both heuristics
We obtain similar results than with the Best-First Search algorithm. Manhattan Distance is a much more accurate heuristic than Misplaced Tiles with much less nodes expanded: 2957<31 605. However, the gap between the number of nodes visited is definitely bigger than with Best-First Search. This time, Manhattan Distance has reduced the number of nodes visited by 10.7. We can see how much better Manhattan Distance is at guiding the search towards the goal. However, the number of moves to reach the goal in the solution path is the same: 26 moves. This shows that even if both heuristics eventually find the optimal solution, Manhattan Distance does it much more efficiently.

## G/Comparison and Analysis

In [13]:
def allAlgorithms(puzzle):
    """
    This function executes multiple search algorithms on the given puzzle to find the solution path and 
    prints the total number of nodes visited and the total number of moves made for each algorithm.

    Args:
        puzzle: the puzzle problem to be solved
    """

    #UCS Algorithm
    print("Uniform Cost Search:\n")
    solutionPath, totalNodesVisited=UniformCostSearch(puzzle)
    totalMoves=len(solutionPath)
    print(f"Total nodes visited: {totalNodesVisited} and Total moves: {totalMoves}\n\n")
    

    #Best-First Search Algorithm
    #Misplaced Tiles
    print("Best-First Search, Misplaced Tiles:\n")
    solutionPath, totalNodesVisited=BestFirstSearch(puzzle,"misplaced")
    totalMoves=len(solutionPath)
    print(f"Total nodes visited: {totalNodesVisited} and Total moves: {totalMoves}\n")

    #Manhattan Distance
    print("Best-First Search, Manhattan Distance:\n")
    solutionPath, totalNodesVisited=BestFirstSearch(puzzle,"manhattan")
    totalMoves=len(solutionPath)
    print(f"Total nodes visited: {totalNodesVisited} and Total moves: {totalMoves}\n")

    #H3
    print("Best-First Search, h3:\n")
    solutionPath, totalNodesVisited=BestFirstSearch(puzzle,"h3")
    totalMoves=len(solutionPath)
    print(f"Total nodes visited: {totalNodesVisited} and Total moves: {totalMoves}\n\n")


    #A* Algorithm
    #Misplaced Tiles
    print("A*, Misplaced Tiles:\n")
    solutionPath, totalNodesVisited=AStar(puzzle,"misplaced")
    totalMoves=len(solutionPath)
    print(f"Total nodes visited: {totalNodesVisited} and Total moves: {totalMoves}\n")

    #Manhattan Distance
    print("A*, Manhattan Distance:\n")
    solutionPath, totalNodesVisited=AStar(puzzle,"manhattan")
    totalMoves=len(solutionPath)
    print(f"Total nodes visited: {totalNodesVisited} and Total moves: {totalMoves}")

allAlgorithms(puzzle)




Uniform Cost Search:

Total nodes visited: 168672 and Total moves: 26


Best-First Search, Misplaced Tiles:

Total nodes visited: 896 and Total moves: 106

Best-First Search, Manhattan Distance:

Total nodes visited: 280 and Total moves: 66

Best-First Search, h3:

Total nodes visited: 345 and Total moves: 84


A*, Misplaced Tiles:

Total nodes visited: 31605 and Total moves: 26

A*, Manhattan Distance:

Total nodes visited: 2957 and Total moves: 26


### Comparison between UCS, Best-First Search and A*
When we compare UCS, Best-First Search, and A*, we can see that the main difference is how they use heuristics and how this affects their results. UCS expands nodes based on the cost of moves, which we called depth in our code and which leads to a very big number of node expansions (168 672) to reach the optimal solution. This algorithm finds the optimal solution (26 moves) which is good but it is way less efficient than the other two algorithms. Indeed, Best-First Search, which only uses heuristics to calculate the path cost of a node, is much more efficient than UCS. It is particularly more efficient with a heuristic like Manhattan Distance which is more accurate: 280 nodes visited compared to Misplaced Tiles: 896 nodes visited. The only problem is that it doesn't always guarantee the optimal solution. We find 106 and 66 moves compared to the optimal solution which is 26. Best-First Search is definitely more efficient than UCS but not optimal. On the other hand, A* sums both the cost of the moves (the depth in our context) and the heuristic value. With the Manhattan Distance heuristic, A* visits only 2957 nodes to find the optimal solution, making it both highly efficient and guaranteed to find the optimal solution. This algorithm takes the best of both UCS and Best-First Search: UCS' optimality and Best-First Search's well guided search. Finally, we can conclude that A* with Manhattan Distance is the most effective in terms of both performance and solution optimality.